# Optimization and Processing of Pandas DataFrames in Chunks for Low Memory Footprint

In this guided project by Dataquest (DQ), I demonstrated the processing of a Pandas DataFrame in chunks and selection of appropriate data types to optimize its memory usage. Processing of datasets can start to become an issue as its file size and memory usage become large. Performance issues can cause longer run times and even code failure due to insufficient memory. 

I worked with a sample dataset from [Lending Club](https://www.lendingclub.com/), an American financial lending company that matches borrowers with investors. The dataset contained information on loans approved from 2007-2011. DQ provided the dataset as `loans_2007.csv`. 

My goal in this project is to illustrate the processing of the dataset using only 10 MB of memory and reduce its overall memory footprint as a Pandas DataFrame. 

## Initial Data Exploration

For initial exploration, I first extracted the first 5 rows of `loans_2007.csv` to check on the number and data types of the columns.

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 99

first_5_rows = pd.read_csv('loans_2007.csv', nrows=5)
first_5_rows

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


The provided dataset has 52 columns, where 21 are of string type and the rest are of numeric type. 

In [2]:
print('Total number of columns:', first_5_rows.columns.size) 
print('Number of string columns:', first_5_rows.select_dtypes(include='object').columns.size)
print('Number of numeric columns:', first_5_rows.select_dtypes(include='number').columns.size)

Total number of columns: 52
Number of string columns: 21
Number of numeric columns: 31


Since I am only limited to 10 MB of memory for data processing, I inspected first the total memory usage of the first 1000 rows of the dataset. 

In [3]:
first_1000_rows = pd.read_csv('loans_2007.csv', nrows=1000)
print('Memory of first 1000 rows, MB =', round(first_1000_rows.memory_usage(deep=True).sum()/2**20,1))

Memory of first 1000 rows, MB = 1.5


With a memory usage of only 1.5 MB for 1000 rows of data, I could further increase the number of rows assigned to a chunk of data to achieve a memory usage of under 5 MB (< 50% of 10 MB). As shown below, I found the appropriate `chunksize` to be 3000 rows which converted the data from `loans_2007.csv` to 15 chunks. The dataset has also a total of 42538 rows.

In [4]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
total_rows_bef = 0
for count, chunk in enumerate(chunk_iter):
    print('Chunk {}, MB = '.format(count), round(chunk.memory_usage(deep=True).sum()/2**20, 1))
    total_rows_bef += len(chunk)

print('\n')
print('Total number of rows = ', total_rows_bef)

Chunk 0, MB =  4.6
Chunk 1, MB =  4.6
Chunk 2, MB =  4.6
Chunk 3, MB =  4.6
Chunk 4, MB =  4.6
Chunk 5, MB =  4.6
Chunk 6, MB =  4.6
Chunk 7, MB =  4.6
Chunk 8, MB =  4.6
Chunk 9, MB =  4.6
Chunk 10, MB =  4.6
Chunk 11, MB =  4.6
Chunk 12, MB =  4.6
Chunk 13, MB =  4.9
Chunk 14, MB =  0.9


Total number of rows =  42538


## Further Data Exploration in Chunks

After selecting a suitable `chunksize` to represent the dataset, I inspected the data types of columns (string or numeric) per chunk. The first 13 chunks of data show 31 numeric and 21 string type columns which confirm my initial result from the first 5 rows of data. However, the last 2 chunks of data show 30 numeric and 22 string type columns. 

In [5]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
numeric_cols = []
string_cols = []
for chunk in chunk_iter:
    numeric_cols.append(chunk.select_dtypes(include='number').columns.size)
    string_cols.append(chunk.select_dtypes(include='object').columns.size)

print(numeric_cols)
print(string_cols)

[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


Further inspection of data columns per chunk shows the transition of the `id` column from numeric to string type at the last 2 chunks. 

In [6]:
object_cols = []
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
for count, chunk in enumerate(chunk_iter):
    if len(object_cols) > 0:
        chunk_cols = chunk.select_dtypes(include='object').columns.tolist()
        if chunk_cols != object_cols:
            print('Overall string columns:', object_cols, "\n")
            print('Chunk {} string columns:'.format(count), chunk_cols, "\n")
    else:
        object_cols = chunk.select_dtypes(include='object').columns.tolist()


Overall string columns: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

Chunk 13 string columns: ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

Overall string columns: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', '

String type columns usually consume more memory than numeric types. For string columns containing a limited number of unique values, I could assign the `category` data type for less memory usage. From the code below, I determined 11 string type columns which contain less than 50 unique values. Except for the `initial_list_status` and `application_type` columns which only have 1 unique value, I found these columns suitable for `category` type representation. 

In [7]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

col_unique_vals = {} 
for chunk in chunk_iter:
    str_chunk = chunk.select_dtypes(include='object')
    str_chunk_cols = str_chunk.columns

    for col in str_chunk_cols:
        unique = str_chunk[col].value_counts()
        
        if col in col_unique_vals:
            col_unique_vals[col].update(unique.index)
        else:
            col_unique_vals[col] = set(unique.index) 
        
for col in col_unique_vals:
    if len(col_unique_vals[col]) < 50:
        print(col, ":", len(col_unique_vals[col]))

term : 2
grade : 7
sub_grade : 35
emp_length : 11
home_ownership : 5
verification_status : 3
loan_status : 9
pymnt_plan : 2
purpose : 14
initial_list_status : 1
application_type : 1


As for the numeric type columns, I determined the columns without missing values for possible conversion to the `interger` type. Only the `id` column has zero missing values. 

In [8]:
missing_vals = []
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in chunk_iter:
    float_chunk = chunk.select_dtypes(include='number')
    
    missing_vals.append(float_chunk.isnull().sum())
    
combined_missing_vals = pd.concat(missing_vals)
combined_missing_vals.groupby(combined_missing_vals.index).sum().sort_values()

id                               0
member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
total_pymnt                      3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
loan_amnt                        3
last_pymnt_amnt                  3
total_rec_prncp                  3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
collection_recovery_fee          3
installment                      3
annual_inc                       7
inq_last_6mths                  32
total_acc                       32
delinq_2yrs                     32
pub_rec                         32
delinq_amnt                     32
open_acc                        32
acc_now_delinq                  32
tax_liens                      108
collections_12_mths_

Before the data modification, I calculated the total memory usage of the dataset to be 65.7 MB. Python also assigned `object` and `float64`  data types to the columns. 

In [9]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

total_memory_bef = []
for chunk in chunk_iter:
    total_memory_bef.append(chunk.memory_usage(deep=True).sum()/2**20)
print('Total memory, MB =', round(sum(total_memory_bef), 1))
print('\n')
chunk.dtypes

Total memory, MB = 65.7




id                             object
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc    

## Data Cleaning and Optimization

To reduce the total memory usage of `loans_2007.csv` as a Pandas DataFrame, I performed the following:

* Parsed the string type columns `issue_d`, `earliest_cr_line`, `last_pymnt_d`, and `last_credit_pull_d` into `datetime` type. These columns portray dates as values. 
* Assigned `category` type to the string type columns `term`, `grade`, `sub_grade`, `emp_length`, `home_ownership`, `verification_status` , `loan_status`, `pymnt_plan`, and `purpose`.
* Removed the rows with missing values. I later compared the total number of rows before and after removal to ensure that a significant amount of information is retained from the dataset. 
* Cleaned the string type columns `int_rate` and `revol_util`, e.g. removed the `%` and white spaces, and converted them into the optimum numeric `float` type. 
* Converted the numeric type columns (predefined as `float64`) into the optimum `float` type.
* Converted the numeric type `id` column into the optimum `integer` type.

Data cleaning and optimization show a 9.7% reduction in data due to the removal of rows with missing values. The assignment of appropriate data types to the columns results to memory savings of 67.5%. I also confirmed the successful assignment of new data types into the columns as shown. 

In [10]:
datetime_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'] # string columns to datetime type
category_cols = {'term':'category', 'grade':'category', 'sub_grade':'category', 
                 'emp_length':'category', 'home_ownership':'category', 
                 'verification_status':'category', 'loan_status':'category', 
                 'pymnt_plan':'category', 'purpose':'category'} # string columns to category type

chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000, parse_dates=datetime_cols, dtype=category_cols)

total_memory_aft = []
total_rows_aft = 0

for chunk in chunk_iter:
    
    # Start of data cleaning, optimization
    chunk = chunk.dropna(axis=0)
    
    # Specific string columns to optimum float type
    chunk['int_rate'] = chunk['int_rate'].str.lstrip().str.rstrip('%') 
    chunk['int_rate'] = pd.to_numeric(chunk['int_rate'], downcast='float')
    chunk['revol_util'] = chunk['revol_util'].str.lstrip().str.rstrip('%') 
    chunk['revol_util'] = pd.to_numeric(chunk['revol_util'], downcast='float')
    
    # Numeric columns to optimum numeric type
    for col in chunk.select_dtypes(include='number').columns:
        if col != 'id':
            chunk[col] = pd.to_numeric(chunk[col], downcast='float')
    chunk['id'] = pd.to_numeric(chunk['id'], downcast='integer')
    
    #  Memory, size of chunk after data cleaning, optimization
    total_memory_aft.append(chunk.memory_usage(deep=True).sum()/2**20)
    total_rows_aft += len(chunk)

print('Before data cleaning, optimization:')
print('Total number of rows:', total_rows_bef)
print('Total memory, MB:', round(sum(total_memory_bef),1))
print('\n')
print('After data cleaning, optimization:')
print('Total number of rows:', total_rows_aft)
print('Total memory, MB:', round(sum(total_memory_aft),1))
print('-------------------------------------------')
print('Data reduction, % =', round(100*(total_rows_bef-total_rows_aft)/total_rows_bef,1))
print('Memory savings, % =', round(100*(sum(total_memory_bef)-sum(total_memory_aft))/sum(total_memory_bef),1))

print('\n', chunk.dtypes)


Before data cleaning, optimization:
Total number of rows: 42538
Total memory, MB: 65.7


After data cleaning, optimization:
Total number of rows: 38433
Total memory, MB: 21.4
-------------------------------------------
Data reduction, % = 9.7
Memory savings, % = 67.5

 id                                     int32
member_id                            float32
loan_amnt                            float32
funded_amnt                          float32
funded_amnt_inv                      float32
term                                category
int_rate                             float32
installment                          float32
grade                               category
sub_grade                           category
emp_title                             object
emp_length                          category
home_ownership                      category
annual_inc                           float32
verification_status                 category
issue_d                       datetime64[ns]
loan_statu

## Conclusion

In this guided project, I processed a dataset in chunks using a limited memory of 10 MB. As a Pandas DataFrame, I also optimized its total memory usage by data cleaning and assignment of appropriate data types. After optimization, I reduced the memory footprint of the dataset by 67.5%. 